Install packages and dependencies needed.

In [ ]:
# imports
!pip install pandas numpy torch transformers yfinance tqdm

Mount Google Drive, and get the path towards the Computational Finance folder that has all the datasets. Then, wee will retrieve the csv files with the data


In [14]:
#google drive imports
from google.colab import drive
drive.mount('/content/drive')

import os

folder_path = '/content/drive/My Drive/BUFN403/UM_Computational_Finance'

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
print(csv_files)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['HAL_2017_2021.csv', 'CBRE_2017_2021.csv', 'CI_2017_2021.csv', 'AZO_2017_2021.csv', 'BMY_2017_2021.csv', 'CMS_2017_2021.csv', 'DHI_2017_2021.csv', 'BA_2017_2021.csv', 'DGX_2017_2021.csv', 'KO_2017_2021.csv', 'BSX_2017_2021.csv', 'KEYS_2017_2021.csv', 'MCO_2017_2021.csv', 'AMZN_2017_2021.csv', 'MKTX_2017_2021.csv', 'JNPR_2017_2021.csv', 'GPN_2017_2021.csv', 'TGT_2017_2021.csv', 'FDX_2017_2021.csv', 'PFG_2017_2021.csv', 'NVDA_2017_2021.csv', 'AAPL_2017_2021.csv', 'EMN_2017_2021.csv', 'RL_2017_2021.csv']


Access the data in all the CSV files, and merge them into one, then we sample n of them, just for the time constraint.

In [15]:
# get the csvs, and merge them
import torch
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

file_names = csv_files
dataframes = [pd.read_csv(folder_path + '/' + file_name) for file_name in file_names]

df_combined = pd.concat(dataframes, ignore_index=True)

df_combined.drop_duplicates(subset=['text'], inplace=True)
df = df_combined.sample(n=2000, random_state=42)
display(df)

,id,sequence,label,date,quarter,year,speaker,title,text,mystery indicator,VADER sentiment,extracted_topics,SASB
17938,BSX-2021-Q3-2021-10-27,97,Answer,2021-10-27,3,2021,Michael F. Mahoney,"Chairman, President & Chief Executive Officer",So it's expanding the awareness when you're re...,0.507413,0.422,"Analyst Update, General News | Opinion",NaN
31457,JNPR-2021-Q4-2022-01-27,8,Remarks,2022-01-27,4,2021,Rami Rahim,Chief Executive Officer & Director,We delivered strong results during the fourth ...,0.408390,0.967,"Analyst Update, Financials",Supply_Chain_Management
3872,CI-2019-Q1-2019-05-02,88,Answer,2019-05-02,1,2019,David Michael Cordani,"President, Chief Executive Officer & Director",First from a growth standpoint at a macro leve...,0.628449,0.900,NaN,Business_Model_Resilience
52570,RL-2017-Q1-2016-08-10,125,Answer,2016-08-10,1,2017,Stefan Larsson,NaN,"Okay. Thank you, David. So when it comes to tr...",NaN,0.796,"Analyst Update, General News | Opinion",NaN
31929,JNPR-2017-Q2-2017-07-25,60,Answer,2017-07-25,2,2017,"Rami Rahim - Juniper Networks, Inc.",NaN,And just to add some color on the first questi...,NaN,0.891,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9370,DHI-2021-Q1-2021-01-26,150,Question,2021-01-26,1,2021,Bill W. Wheat,"CFO, Principal Accounting Officer & Executive VP",So just wondering if there was anything specif...,0.176590,0.891,NaN,NaN
23343,MCO-2019-Q3-2019-10-30,160,Answer,2019-10-30,3,2019,Robert Scott Fauber,Chief Operating Officer,"So, in general, I'd expect issuance to continu...",1.390854,0.273,NaN,NaN
13626,DGX-2021-Q4-2022-02-03,22,Remarks,2022-02-03,4,2021,Stephen H. Rusckowski,"Chairman, President & Chief Executive Officer","Now turning to our results, we closed out 2021...",0.092257,0.422,"Analyst Update, Financials",NaN
4560,AZO-2018-Q1-2017-12-05,122,Answer,2017-12-05,1,2018,William C. Rhodes,"Chairman, President & Chief Executive Officer","Yes. I guess I would say, it's very similar to...",0.344653,0.542,NaN,NaN


In the below cell, we use the apply method on df to calculate the average stock price for each row based on the company_ticker and year, storing the result in a new column named avg_price. This step leverages the get_price function. Then,  we calculate the year-over-year price change for each stock ticker. This is achieved by grouping the DataFrame by company_ticker and then applying the diff method on the avg_price column to get the difference in average price between consecutive years. The result is stored in a new column named price_change, which is the outcome variable that we are trying to predict.

In [16]:
import yfinance as yf
import pandas as pd

# method to get ticker from id
def extract_ticker(id_str):
    return id_str.split('-')[0]
df['company_ticker'] = df['id'].apply(extract_ticker)

# method to get price for that ticker, and gets the average of it, and reports that as the stock price
def get_price(ticker, year):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="max")
        yearly_prices = hist.loc[hist.index.year == year, 'Close']
        if not yearly_prices.empty:
            avg_price = yearly_prices.mean()
            return avg_price
        else:
            return None
    except Exception as e:
        print(f"Error retrieving price for {ticker}: {e}")
        return None

# type checking, and updating
df['year'] = df['year'].astype(int)
df['avg_price'] = df.apply(lambda x: get_price(x['company_ticker'], x['year']), axis=1)

df.sort_values(by=['company_ticker', 'year'], inplace=True)
df['price_change'] = df.groupby('company_ticker')['avg_price'].diff()

display(df[['company_ticker', 'year', 'avg_price', 'price_change']])


,company_ticker,year,avg_price,price_change
50287,AAPL,2017,35.299757,NaN
50277,AAPL,2017,35.299757,6.839099e-07
49239,AAPL,2017,35.299757,-6.383159e-07
49327,AAPL,2017,35.299757,3.571529e-07
50283,AAPL,2017,35.299757,1.215840e-07
...,...,...,...,...
37636,TGT,2021,210.910929,1.513769e-06
36963,TGT,2021,210.910930,4.844060e-07
36417,TGT,2021,210.910930,3.633045e-07
37904,TGT,2021,210.910930,-3.027537e-07


This code segments a pandas DataFrame to categorize numerical price changes into five distinct groups based on set thresholds, then generates features from various columns through one-hot encoding and combines them for predictive modeling. Using these features, it trains a logistic regression model to forecast price change categories, employing standardized data and evaluating model performance via training and testing accuracy metrics. Lastly, it outputs the model's coefficients for each category, providing insight into the significance of each feature in predicting price change categories.

In [21]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# method to bin everything
def bin_price_change(x):
    # Not these cut offs are arbitrary, and can be modified
    if x > 10:
        return 'Significant gain'
    elif x > 1:
        return 'Moderate gain'
    elif x > -1:
        return 'No change'
    elif x > -10:
        return 'Moderate loss'
    else:
        return 'Significant loss'

# getting price change
df['price_change_category'] = df['price_change'].apply(bin_price_change)

# see distribution
category_counts = df['price_change_category'].value_counts()

print(category_counts)

# Given code for generating features
topics = df['extracted_topics'].str.get_dummies(sep=',')
company_dummies = pd.get_dummies(df['company_ticker'], prefix='company')

# Ensure 'mystery indicator' and 'VADER sentiment' are not dropped due to NaNs
df['mystery indicator'].fillna(df['mystery indicator'].mean(), inplace=True)
df['VADER sentiment'].fillna(df['VADER sentiment'].mean(), inplace=True)

# pick inpit features
X = pd.concat([topics, df['mystery indicator'], df['VADER sentiment'], company_dummies], axis=1)
X.dropna(axis=1, how='any', inplace=True)
# output features
y = df['price_change_category']

X = X.loc[y.index]

# scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# model training and fitting
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')

model.fit(X_train_scaled, y_train)

print('Training accuracy:', model.score(X_train_scaled, y_train))
print('Testing accuracy:', model.score(X_test_scaled, y_test))

feature_names = X.columns
coefficients = model.coef_
class_labels = model.classes_

print("\nModel coefficients:")
for index, class_label in enumerate(class_labels):
    print(f"Class '{class_label}':")
    for coef, feature in zip(coefficients[index], feature_names):
        print(f"{feature}: {coef}")
    print("\n")


No change           1885
Significant gain      51
Significant loss      31
Moderate gain         23
Moderate loss         10
Name: price_change_category, dtype: int64
Training accuracy: 0.945
Testing accuracy: 0.9325

Model coefficients:
Class 'Moderate gain':
 Company | Product News: -0.10189564659763814
 Dividend: -0.000578605240869737
 Earnings: -0.07033799819316593
 Energy | Oil: -0.0018254561886855572
 Fed | Central Banks: -0.000578605240869737
 Financials: 0.1391019694900227
 General News | Opinion: -0.13613438951999346
 Legal | Regulation: -0.07258356126084231
 M&A | Investments: -0.040386763036622725
 Macro: -0.09673306801342627
 Personnel Change: -0.027488428544094324
 Politics: -0.037892789950548685
 Stock Commentary: -0.027204971697839888
 Treasuries | Corporate Debt: -0.05018197024362037
Analyst Update: -0.18553306970400354
mystery indicator: 0.1817324474865315
VADER sentiment: 0.15105511662117949
company_AAPL: 0.3682030238476536
company_AMZN: -0.11444436012900096
company_A